# Functionality of the `cad` module

The `cad` module allows for the construction of a _cylindrical algebraic decomposition_ for polynomial systems of a special type, as described in our paper. This module in particular can construct CADs of simplices, glue disjoint simplices together, and extend these to a simplex-extensible function of the form $f^* := g_0 + \sum_1^n x_i g_i$. As we showed in the paper, this function is simplex-extensible if and only if each of $\{g_i\}_0^n$ are sign-invariant over the cells of the given simplex CAD. We leave it to the user to check this. However, we showed in the paper that this property holds for inequalities arising from the Markov analyses implemented in the `markov` module. This notebook demonstrates its functionality.

## Imports

In [4]:
from markovag import cad
from sympy import var, Interval, FiniteSet

var('a,b,c,d,e,f,x,x_1,x_2,x_3')

(a, b, c, d, e, f, x, x_1, x_2, x_3)

## Foundational classes

This module introduces the `CadNode` class, which is the foundation for all CAD construction. The CAD data structure is essentially a tree, where each node holds a variable and a sample point. In our class, one passes a variable and a condition, which is either a `sympy.Interval` or a `sympy.FinitePoint`, based on the type of cell. Based on this, the class automatically computes a sample point. As well, it propagates sample points from its ancestors, if a parent is given. In order to start a CAD tree, there must be a `CadRootNode` object as the root. This does not add any information to the tree, it is just required for manipulation. We will see this later when we build CADs.

In [3]:
node = cad.CadNode(x_1, Interval(0,1))
print(node.var, node.sample)

x_1 1/2


In [4]:
node1 = cad.CadNode(x_2, Interval(0, x_1), parent=node)
print(node1.var, node1.sample)

x_2 1/4


Observe that the sample point for `node1` has been accurately computed, as this cell is being lifted over the cell where $x_1 \in (0,1)$.

## The Simplex CAD

In the paper, we show that there is an easy characterization of the simplex CAD. It can be worked out by hand pretty easily... although may become tedious if the number of variables grows. However, generating it on a computer takes some care to propagate equalities and simplify expressions. The `simplex_cad()` function takes a list of variables, and makes a simplex CAD of them. The variables appear in the tree in the order in which they are passed. There is also an optional parameter `rhs`, which can take a non-negative value less than 1, as this is necessary for some Markov analyses. The function returns the root node of the tree.

In [104]:
simplex = cad.simplex_cad([a,b,c])

We will discuss printing trees in more detail further down. For now, we will use the `cad_print()` function to render it.

In [105]:
cad.cad_print(simplex)

root
├──  $a = 0$
│   ├──  $b = 0$
│   │   └──  $c = 1$
│   ├──  $0 < b < 1$
│   │   └──  $c = 1 - b$
│   └──  $b = 1$
│       └──  $c = 0$
├──  $0 < a < 1$
│   ├──  $b = 0$
│   │   └──  $c = 1 - a$
│   ├──  $0 < b < 1 - a$
│   │   └──  $c = - a - b + 1$
│   └──  $b = 1 - a$
│       └──  $c = 0$
└──  $a = 1$
    └──  $b = 0$
        └──  $c = 0$


This is a pretty small CAD which makes sense. We can also change the `rhs` to another value.

In [106]:
simplex = cad.simplex_cad([a,b,c], rhs=0.5)
cad.cad_print(simplex)

root
├──  $a = 0$
│   ├──  $b = 0$
│   │   └──  $c = 0.5$
│   ├──  $0 < b < 0.5$
│   │   └──  $c = 0.5 - b$
│   └──  $b = 0.5$
│       └──  $c = 0$
├──  $0 < a < 0.5$
│   ├──  $b = 0$
│   │   └──  $c = 0.5 - a$
│   ├──  $0 < b < 0.5 - a$
│   │   └──  $c = - a - b + 0.5$
│   └──  $b = 0.5 - a$
│       └──  $c = 0$
└──  $a = 0.5$
    └──  $b = 0$
        └──  $c = 0$


We can then glue multiple simplices together. The function `glue_simplex_cads()` takes a list (of the roots) of simplex CADs. They should be disjoint. The user should check this before calling it. They will be glued in order. It will return the root of the first simplex CAD. Because of the way the `anytree` package works, the first simplex tree will be modified. If you don't want this, you can `import copy` and use `copy.deepcopy()` first. 

In [107]:
simplex_1 = cad.simplex_cad([a,b,c])
simplex_2 = cad.simplex_cad([d,e,f], rhs=0.5)

glued = cad.glue_simplex_cads([simplex_1, simplex_2])

cad.cad_print(glued)

root
├──  $a = 0$
│   ├──  $b = 0$
│   │   └──  $c = 1$
│   │       ├──  $d = 0$
│   │       │   ├──  $e = 0$
│   │       │   │   └──  $f = 0.5$
│   │       │   ├──  $0 < e < 0.5$
│   │       │   │   └──  $f = 0.5 - e$
│   │       │   └──  $e = 0.5$
│   │       │       └──  $f = 0$
│   │       ├──  $0 < d < 0.5$
│   │       │   ├──  $e = 0$
│   │       │   │   └──  $f = 0.5 - d$
│   │       │   ├──  $0 < e < 0.5 - d$
│   │       │   │   └──  $f = - d - e + 0.5$
│   │       │   └──  $e = 0.5 - d$
│   │       │       └──  $f = 0$
│   │       └──  $d = 0.5$
│   │           └──  $e = 0$
│   │               └──  $f = 0$
│   ├──  $0 < b < 1$
│   │   └──  $c = 1 - b$
│   │       ├──  $d = 0$
│   │       │   ├──  $e = 0$
│   │       │   │   └──  $f = 0.5$
│   │       │   ├──  $0 < e < 0.5$
│   │       │   │   └──  $f = 0.5 - e$
│   │       │   └──  $e = 0.5$
│   │       │       └──  $f = 0$
│   │       ├──  $0 < d < 0.5$
│   │       │   ├──  $e = 0$
│   │       │   │   └──  $f = 0.5 - d$
│   │

But note that the first simplex that was passed has also been modified! The variables `glued` and `simplex_1` point to the same root.

In [108]:
cad.cad_print(simplex_1)

root
├──  $a = 0$
│   ├──  $b = 0$
│   │   └──  $c = 1$
│   │       ├──  $d = 0$
│   │       │   ├──  $e = 0$
│   │       │   │   └──  $f = 0.5$
│   │       │   ├──  $0 < e < 0.5$
│   │       │   │   └──  $f = 0.5 - e$
│   │       │   └──  $e = 0.5$
│   │       │       └──  $f = 0$
│   │       ├──  $0 < d < 0.5$
│   │       │   ├──  $e = 0$
│   │       │   │   └──  $f = 0.5 - d$
│   │       │   ├──  $0 < e < 0.5 - d$
│   │       │   │   └──  $f = - d - e + 0.5$
│   │       │   └──  $e = 0.5 - d$
│   │       │       └──  $f = 0$
│   │       └──  $d = 0.5$
│   │           └──  $e = 0$
│   │               └──  $f = 0$
│   ├──  $0 < b < 1$
│   │   └──  $c = 1 - b$
│   │       ├──  $d = 0$
│   │       │   ├──  $e = 0$
│   │       │   │   └──  $f = 0.5$
│   │       │   ├──  $0 < e < 0.5$
│   │       │   │   └──  $f = 0.5 - e$
│   │       │   └──  $e = 0.5$
│   │       │       └──  $f = 0$
│   │       ├──  $0 < d < 0.5$
│   │       │   ├──  $e = 0$
│   │       │   │   └──  $f = 0.5 - d$
│   │

## Extending to a simplex-extensible function

The procedure discussed in the paper is that we should compute the simplex CADs, glue them together, _then_ lift on the remaining variables. This is what the function `simplex_extensible_cad()` does. To use it, we pass the root of a simplex CAD (which may be many simplices glued together), a function $g_0$ of the $\alpha$-type variables, a list of functions $g_i$ of the $\alpha$-type variables, and then a list of $x$-type variables. See our paper for the definitions.

Suppose we have the following system: $f^* := ax_1 + bx_2 - 1 \geq 0$ such that $a+b=1$, and $a,b \in [0,1]$, and $x_1,x_2 \geq 0$. We can decompose this into:
$$
f^* = \underbrace{a}_{g_1} x_1 + \underbrace{b}_{g_2} x_2 + \underbrace{-1}_{g_0}
$$

As each of $g_0, g_1, g_2$ are invariant over the unit simplex, $f^*$ is _simplex-extensible_, in the words of Maaz 2024. Therefore, we can construct the CAD in a simple way.

In [110]:
tree = cad.simplex_cad([a,b])
tree = cad.simplex_extensible_cad(tree, -1, [1,1], [x_1, x_2])

cad.cad_print(tree)

root
├──  $a = 0$
│   └──  $b = 1$
│       ├──  $x_1 = 0$
│       │   ├──  $x_2 = 1$
│       │   └──  $x_2 > 1$
│       ├──  $0 < x_1 < 1$
│       │   ├──  $x_2 = 1 - x_{1}$
│       │   └──  $x_2 > 1 - x_{1}$
│       ├──  $x_1 = 1$
│       │   ├──  $x_2 = 0$
│       │   └──  $x_2 > 0$
│       └──  $x_1 > 1$
│           ├──  $x_2 = 0$
│           └──  $x_2 > 0$
├──  $0 < a < 1$
│   └──  $b = 1 - a$
│       ├──  $x_1 = 0$
│       │   ├──  $x_2 = 1$
│       │   └──  $x_2 > 1$
│       ├──  $0 < x_1 < 1$
│       │   ├──  $x_2 = 1 - x_{1}$
│       │   └──  $x_2 > 1 - x_{1}$
│       ├──  $x_1 = 1$
│       │   ├──  $x_2 = 0$
│       │   └──  $x_2 > 0$
│       └──  $x_1 > 1$
│           ├──  $x_2 = 0$
│           └──  $x_2 > 0$
└──  $a = 1$
    └──  $b = 0$
        ├──  $x_1 = 0$
        │   ├──  $x_2 = 1$
        │   └──  $x_2 > 1$
        ├──  $0 < x_1 < 1$
        │   ├──  $x_2 = 1 - x_{1}$
        │   └──  $x_2 > 1 - x_{1}$
        ├──  $x_1 = 1$
        │   ├──  $x_2 = 0$
        │   └──  

This is a pretty simple function. Based on the theory developed in the paper, the function can create the _possible_ cells for the CAD, and then afterwards the tree is pruned according to if the defining inequality $f^* \geq 0$ holds. We can extend this to more complicated simplex-extensible functions. Suppose now $f^* := -10 + abx_1 + (a^2 + b^2) x_2$

In [147]:
tree = cad.simplex_cad([a,b])
tree = cad.simplex_extensible_cad(tree, -10, [a*b, a**2 + b**2], [x_1, x_2])

cad.cad_print(tree)

root
├──  $a = 0$
│   └──  $b = 1$
│       ├──  $x_1 = 0$
│       │   ├──  $x_2 = 10$
│       │   └──  $x_2 > 10$
│       └──  $x_1 > 0$
│           ├──  $x_2 = 10$
│           └──  $x_2 > 10$
├──  $0 < a < 1$
│   └──  $b = 1 - a$
│       ├──  $x_1 = 0$
│       │   ├──  $x_2 = \frac{10}{a^{2} + \left(1 - a\right)^{2}}$
│       │   └──  $x_2 > \frac{10}{a^{2} + \left(1 - a\right)^{2}}$
│       ├──  $0 < x_1 < \frac{10}{a \left(1 - a\right)}$
│       │   ├──  $x_2 = \frac{- a x_{1} \cdot \left(1 - a\right) + 10}{a^{2} + \left(1 - a\right)^{2}}$
│       │   └──  $x_2 > \frac{- a x_{1} \cdot \left(1 - a\right) + 10}{a^{2} + \left(1 - a\right)^{2}}$
│       ├──  $x_1 = \frac{10}{a \left(1 - a\right)}$
│       │   ├──  $x_2 = 0$
│       │   └──  $x_2 > 0$
│       └──  $x_1 > \frac{10}{a \left(1 - a\right)}$
│           ├──  $x_2 = 0$
│           └──  $x_2 > 0$
└──  $a = 1$
    └──  $b = 0$
        ├──  $x_1 = 0$
        │   ├──  $x_2 = 10$
        │   └──  $x_2 > 10$
        └──  $x_1 > 0$
 

Observe the tree pruning in action! In the cell $0 < a < 1 \land b = 1 - a \land x_1 = 0$, a _possible_ cell for $x_2$, based on the roots of the relevant projection factor, is $x_2 = 0$ (and $0 < x_2 < \frac{10}{a^{2} + \left(1 - a\right)^{2}}$). But, these cells are deleted based on the evaluation of the defining inequality.

Maybe we only care about when $a$ lies in the open interval, because perhaps the endpoints are too extreme.

In [148]:
cad.cad_print(tree.children[1])

 $0 < a < 1$
└──  $b = 1 - a$
    ├──  $x_1 = 0$
    │   ├──  $x_2 = \frac{10}{a^{2} + \left(1 - a\right)^{2}}$
    │   └──  $x_2 > \frac{10}{a^{2} + \left(1 - a\right)^{2}}$
    ├──  $0 < x_1 < \frac{10}{a \left(1 - a\right)}$
    │   ├──  $x_2 = \frac{- a x_{1} \cdot \left(1 - a\right) + 10}{a^{2} + \left(1 - a\right)^{2}}$
    │   └──  $x_2 > \frac{- a x_{1} \cdot \left(1 - a\right) + 10}{a^{2} + \left(1 - a\right)^{2}}$
    ├──  $x_1 = \frac{10}{a \left(1 - a\right)}$
    │   ├──  $x_2 = 0$
    │   └──  $x_2 > 0$
    └──  $x_1 > \frac{10}{a \left(1 - a\right)}$
        ├──  $x_2 = 0$
        └──  $x_2 > 0$


Note that the cells for $x_2$ here could probably be simplified if we want for printing. For example, if we have sibling cells $x_2>0$ and $x_2=0$, we could just write $x_2 \geq 0$. I haven't figured out a good way to do this automatically. Later we will discuss tree rendering.

Let's say we have no simplex constraints, and there are no $\alpha$-type variables. Then, the function still works, although in this case the CAD is quite simple and could probably be inferred by hand. You can pass either an empty simplex, i.e., a CAD with only the root node, or you can pass `None`.

In [149]:
# empty simplex
tree = cad.simplex_cad([])
tree = cad.simplex_extensible_cad(tree, -10, [1, 1], [x_1, x_2])

cad.cad_print(tree)

# or, pass None
tree = cad.simplex_extensible_cad(None, -10, [1, 1], [x_1, x_2])

cad.cad_print(tree)

root
├──  $x_1 = 0$
│   ├──  $x_2 = 10$
│   └──  $x_2 > 10$
├──  $0 < x_1 < 10$
│   ├──  $x_2 = 10 - x_{1}$
│   └──  $x_2 > 10 - x_{1}$
├──  $x_1 = 10$
│   ├──  $x_2 = 0$
│   └──  $x_2 > 0$
└──  $x_1 > 10$
    ├──  $x_2 = 0$
    └──  $x_2 > 0$
root
├──  $x_1 = 0$
│   ├──  $x_2 = 10$
│   └──  $x_2 > 10$
├──  $0 < x_1 < 10$
│   ├──  $x_2 = 10 - x_{1}$
│   └──  $x_2 > 10 - x_{1}$
├──  $x_1 = 10$
│   ├──  $x_2 = 0$
│   └──  $x_2 > 0$
└──  $x_1 > 10$
    ├──  $x_2 = 0$
    └──  $x_2 > 0$


Of course, we can also work with glued simplex CADs.

In [179]:
cad.cad_print(simplex_1)

root
├──  $a = 0$
│   └──  $b = 1$
│       ├──  $c = 0$
│       │   └──  $d = 0.5$
│       ├──  $0 < c < 0.5$
│       │   └──  $d = 0.5 - c$
│       └──  $c = 0.5$
│           └──  $d = 0$
├──  $0 < a < 1$
│   └──  $b = 1 - a$
│       ├──  $c = 0$
│       │   └──  $d = 0.5$
│       ├──  $0 < c < 0.5$
│       │   └──  $d = 0.5 - c$
│       └──  $c = 0.5$
│           └──  $d = 0$
└──  $a = 1$
    └──  $b = 0$
        ├──  $c = 0$
        │   └──  $d = 0.5$
        ├──  $0 < c < 0.5$
        │   └──  $d = 0.5 - c$
        └──  $c = 0.5$
            └──  $d = 0$


In [187]:
simplex_1 = cad.simplex_cad([a,b])
simplex_2 = cad.simplex_cad([c,d], rhs=0.5)

simplex_1 = cad.glue_simplex_cads([simplex_1, simplex_2])
tree = cad.simplex_extensible_cad(simplex_1, -10, [a*b+5, c*d + a], [x_1, x_2])

cad.cad_print(tree)

root
├──  $a = 0$
│   └──  $b = 1$
│       ├──  $c = 0$
│       │   └──  $d = 0.5$
│       │       ├──  $x_1 = 2$
│       │       │   ├──  $x_2 = 0$
│       │       │   └──  $x_2 > 0$
│       │       └──  $x_1 > 2$
│       │           ├──  $x_2 = 0$
│       │           └──  $x_2 > 0$
│       ├──  $0 < c < 0.5$
│       │   └──  $d = 0.5 - c$
│       │       ├──  $x_1 = 0$
│       │       │   ├──  $x_2 = \frac{10}{c \left(0.5 - c\right)}$
│       │       │   └──  $x_2 > \frac{10}{c \left(0.5 - c\right)}$
│       │       ├──  $0 < x_1 < 2$
│       │       │   ├──  $x_2 = \frac{10 - 5 x_{1}}{c \left(0.5 - c\right)}$
│       │       │   └──  $x_2 > \frac{10 - 5 x_{1}}{c \left(0.5 - c\right)}$
│       │       ├──  $x_1 = 2$
│       │       │   ├──  $x_2 = 0$
│       │       │   └──  $x_2 > 0$
│       │       └──  $x_1 > 2$
│       │           ├──  $x_2 = 0$
│       │           └──  $x_2 > 0$
│       └──  $c = 0.5$
│           └──  $d = 0$
│               ├──  $x_1 = 2$
│               │   ├─

If you already have the function $f^*$, there is a utility function that will extract the $g$ functions from it. It returns a list, the first element being $g_0$, and the remaining being $g_i$ matching the $x_i$.

In [189]:
cad.make_g_functions(a*x_1 + b*x_2 - 10*a*b, xvars=[x_1, x_2])

[-10*a*b, a*x_1, b*x_2]

## Rendering utilities

There are two helpful functions given to help with rendering CAD trees. The first one is for printing in your jupyter notbeook or console. Simply pass your root to `cad_print()`. It will print the conditions of each cell in LaTeX.

In [191]:
tree = cad.simplex_cad([a,b])
cad.cad_print(tree)

root
├──  $a = 0$
│   └──  $b = 1$
├──  $0 < a < 1$
│   └──  $b = 1 - a$
└──  $a = 1$
    └──  $b = 0$


You can also set `print_sample=True` if you want the sample point value printed after each node.

In [194]:
cad.cad_print(tree, print_sample=True)

root
├──  $a = 0$, sample=0
│   └──  $b = 1$, sample=1
├──  $0 < a < 1$, sample=1/2
│   └──  $b = 1 - a$, sample=1/2
└──  $a = 1$, sample=1
    └──  $b = 0$, sample=0


You can also specify a `defining_poly`, which is an expression that you want to evaluate at each leaf. This will add a single node to each existing leaf which will display the value of the given polynomial evaluated at the sample points. This can help give an indication of how the polynomial behaves over the tree.

In [196]:
cad.cad_print(tree, print_sample=True, defining_poly=a+b)

root
├──  $a = 0$, sample=0
│   └──  $b = 1$, sample=1
│       └──  $f^* = 1$, sample=1
├──  $0 < a < 1$, sample=1/2
│   └──  $b = 1 - a$, sample=1/2
│       └──  $f^* = 1$, sample=1
└──  $a = 1$, sample=1
    └──  $b = 0$, sample=0
        └──  $f^* = 1$, sample=1


As expected, $a + b = 1$ always.

The other printing function is `cad_latex()`, which takes the root of a tree and converts it into LaTeX code, using the forest package.

In [207]:
tree = cad.simplex_cad([a,b,c])
print(cad.cad_latex(tree))

\begin{forest}
[{}, phantom
[{$a = 0$}
	[{$b = 0$}
		[{$c = 1$}]
	]
	[{$0 < b < 1$}
		[{$c = 1 - b$}]
	]
	[{$b = 1$}
		[{$c = 0$}]
	]
]
[{$0 < a < 1$}
	[{$b = 0$}
		[{$c = 1 - a$}]
	]
	[{$0 < b < 1 - a$}
		[{$c = - a - b + 1$}]
	]
	[{$b = 1 - a$}
		[{$c = 0$}]
	]
]
[{$a = 1$}
	[{$b = 0$}
		[{$c = 0$}]
	]
]
]
\end{forest}


You can also optionally specify `file_path`, to write it to a TeX file.

In [208]:
tree = cad.simplex_cad([a,b,c])
cad.cad_latex(tree, file_path='example_cad.tex')

'\\begin{forest}\n[{}, phantom\n[{$a = 0$}\n\t[{$b = 0$}\n\t\t[{$c = 1$}]\n\t]\n\t[{$0 < b < 1$}\n\t\t[{$c = 1 - b$}]\n\t]\n\t[{$b = 1$}\n\t\t[{$c = 0$}]\n\t]\n]\n[{$0 < a < 1$}\n\t[{$b = 0$}\n\t\t[{$c = 1 - a$}]\n\t]\n\t[{$0 < b < 1 - a$}\n\t\t[{$c = - a - b + 1$}]\n\t]\n\t[{$b = 1 - a$}\n\t\t[{$c = 0$}]\n\t]\n]\n[{$a = 1$}\n\t[{$b = 0$}\n\t\t[{$c = 0$}]\n\t]\n]\n]\n\\end{forest}'

You can then easily include it in as a figure in your LaTeX document as follows:

```
\begin{figure}
        \centering
        \input{example_cad}
\end{figure}
```